In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import jaccard_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, confusion_matrix

from pystruct.datasets import load_letters
from pystruct.models import ChainCRF
from pystruct.learners import FrankWolfeSSVM, OneSlackSSVM, NSlackSSVM, StructuredPerceptron

import pandas as pd

from tqdm.notebook import tqdm

from sklearn.model_selection import KFold

from nltk import ngrams

import warnings
warnings.filterwarnings('error')

def join_txt(text): return np.asarray("-".join(text),dtype=object)

## Feature Extraction

In [ ]:
frequencies = pd.read_csv("./Anchor_pos_data/mhcflurry.ba.frequency_matrices.csv")
frequencies = frequencies[(frequencies['cutoff_fraction'] == 0.01)]
frequencies['X'] = np.zeros(frequencies.shape[0])
frequencies_alleles = pd.unique(frequencies['allele'])

In [ ]:
frequencies

In [ ]:
dataset = pd.read_csv("./Anchor_pos_data/Template_information.csv")
dataset = dataset[(dataset['peptide_length'] > 7) & (dataset['MHC']).isin(frequencies_alleles)]
dataset = dataset[~dataset['pdb_code'].str.contains("_B.pdb")]
dataset["pdb_code"] = dataset["pdb_code"].str.replace("_A","")
dataset = dataset[['MHC', 'peptide', 'peptide_length', 'anchors', 'NetMHCpan4.1_anchors']].drop_duplicates(keep="first")
MHC_list = dataset['MHC'].tolist()
peptide_list = dataset['peptide'].tolist()
labels_list = dataset['anchors'].tolist()
NetMHCpan_list = dataset['NetMHCpan4.1_anchors'].tolist()
lengths_list = dataset['peptide_length'].tolist()

Change feature extraction features at will:

In [ ]:
# Feature Extraction parameters
windows_size = 1
feature_size = 3

In [ ]:
def extract_positions(position, peptide, feature_size):
    scale = range(-int(np.floor(feature_size/2)), int(np.floor(feature_size/2)) + 1)
    position_list = []
    append_left = 0
    append_right = 0
    for x in scale:
        if (position + x < 1):
            position_list.append(0)
            append_left += 1
        elif (position + x > len(peptide)):
            position_list.append(0)
            append_right += 1
        else:
            position_list.append(position + x)
    return position_list, append_left, append_right

wind = int(np.floor(windows_size/2))
all_features_list = []
all_labels_list = []
all_labels_NetMHCpan_list = []
for i, peptide in enumerate(tqdm(peptide_list)):
    
    aa_neighbors = [''.join(w) for w in list(ngrams(peptide, windows_size, pad_left=True, pad_right=True, 
                                                    left_pad_symbol='X', 
                                                    right_pad_symbol='X'))][wind:]
    aa_neighbors = aa_neighbors[:len(aa_neighbors) - wind]
    
    sub_seq_feature_list = []
    for k, sub_seq in enumerate(aa_neighbors):
    
        # Break subsequence down to amino_acids
        pep_sequence = list(sub_seq)
        
        # For each amino-acid, extract frequencies and important positions
        feature_list = []
        for j, amino_acid in enumerate(pep_sequence):
        
            freq_features = frequencies[(frequencies['allele'] == MHC_list[i]) & (frequencies['length'] == lengths_list[i])]
            position_list, append_left, append_right = extract_positions(j + k + 1 - wind, peptide, feature_size)
            freq_features = freq_features[freq_features['position'].isin(position_list)][amino_acid].values
            freq_features = np.hstack([np.zeros(append_left), freq_features, np.zeros(append_right)])
            feature_list.append(freq_features) 
        
        features = np.array(np.hstack(feature_list))
        #posn_feature = (k == 0)
        #posnplus1_feature = (k == 1)
        #posc_feature = (k == len(peptide) - 1) # Get those out!
        #features = np.hstack([features, [posn_feature], [posnplus1_feature], [posc_feature]])
        sub_seq_feature_list.append(features)
    all_features_list.append(np.array(sub_seq_feature_list, dtype=np.float))
    
    labels_int_list = [int(x) - 1 for x in labels_list[i].split(',')]
    labels = np.zeros(len(peptide), dtype=int)
    labels[labels_int_list] = 1
    all_labels_list.append(labels)
    
    labels_NetMHCpan_int_list = [int(x) - 1 for x in NetMHCpan_list[i].split(',')]
    labels_NetMHCpan = np.zeros(len(peptide), dtype=int)
    labels_NetMHCpan[labels_NetMHCpan_int_list] = 1
    all_labels_NetMHCpan_list.append(labels_NetMHCpan)
    
X = np.array(all_features_list, dtype=object)
y = np.array(all_labels_list, dtype=object)
y_NetMHCpan = np.array(all_labels_NetMHCpan_list, dtype=object)

In [ ]:
def correct_predictions(y_test, preds):
    counter = 0
    correct = 0
    for i in range(y_test.shape[0]):
        correct += np.array_equal(y_test[i], preds[counter:(counter + y_test[i].shape[0])])
        counter += y_test[i].shape[0]
        
    # print(str(correct) + "/" + str(y_test.shape[0]) + " correct predictions!")
    return correct

In [ ]:
def correct_rf_predictions(y_test, preds, prob_preds, peptide_list, MHC_list, X):
    counter = 0
    correct = 0
    for i in range(y_test.shape[0]):
        correct += np.array_equal(y_test[i], preds[counter:(counter + y_test[i].shape[0])])
        print(peptide_list[i] + ' ' + MHC_list[i])
        print(y_test[i])
        print(preds[counter:(counter + y_test[i].shape[0])])
        print(prob_preds[counter:(counter + y_test[i].shape[0])])
        print(X[i])
        counter += y_test[i].shape[0]

In [ ]:
def jaccard_predictions(y_test, preds):
    counter = 0
    correct = 0
    for i in range(y_test.shape[0]):
        correct += jaccard_score(y_test[i], preds[counter:(counter + y_test[i].shape[0])])
        counter += y_test[i].shape[0]

    # print(str(correct) + "/" + str(y_test.shape[0]) + " correct predictions!")
    return correct

In [ ]:
def youden_func(y_true, y_score):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    spec = 1 - fpr
    sens = tpr
    idx = np.argmax(spec + sens -1)
    return thresholds[idx]

def abs_d_sens_spec_func(y_true, y_score):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    spec = 1 - fpr
    sens = tpr
    idx = np.argmax(np.abs(sens - spec))
    return thresholds[idx]

def roc01_func(y_true, y_score):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    spec = 1 - fpr
    sens = tpr
    idx = np.argmax(np.sqrt((1 - sens)**2 + (1 - spec)**2))
    return thresholds[idx]

def accuracy_func(y_true, y_score):
    
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    accuracy_scores = []
    for thresh in thresholds:
        accuracy_scores.append(accuracy_score(y_true, [m > thresh for m in y_score]))

    accuracies = np.array(accuracy_scores)
    max_accuracy = accuracies.max() 
    max_accuracy_threshold = thresholds[accuracies.argmax()]
    return max_accuracy_threshold

def cohen_kappa_func(y_true, y_score):
    
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    accuracy_scores = []
    for thresh in thresholds:
        accuracy_scores.append(cohen_kappa_score(y_true, [m > thresh for m in y_score]))

    accuracies = np.array(accuracy_scores)
    max_accuracy = accuracies.max() 
    max_accuracy_threshold = thresholds[accuracies.argmax()]
    return max_accuracy_threshold

def f1_func(y_true, y_score):
    
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    accuracy_scores = []
    for thresh in thresholds:
        accuracy_scores.append(f1_score(y_true, [m > thresh for m in y_score]))

    accuracies = np.array(accuracy_scores)
    max_accuracy = accuracies.max() 
    max_accuracy_threshold = thresholds[accuracies.argmax()]
    return max_accuracy_threshold

def DOP_func(y_true, y_score):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    accuracy_scores = []
    for thresh in thresholds:
        pred_values = [m > thresh for m in y_score]
        CM = confusion_matrix(y_true,pred_values)
        TN = CM[0][0]
        FN = CM[1][0] 
        TP = CM[1][1]
        FP = CM[0][1]
        try:
            PPV = TP/(TP + FP)
            NPV = TN/(TN + FN)
            Sens = TP/(TP + FN)
            Spec = TN/(TN + FP)
            accuracy_scores.append(np.sqrt((1 - Sens)**2 + (1 - Spec)**2 + (1 - PPV)**2 + (1 - NPV)**2))
        except RuntimeWarning:
            accuracy_scores.append(0)
                               
    accuracies = np.array(accuracy_scores)
    max_accuracy = accuracies.max() 
    max_accuracy_threshold = thresholds[accuracies.argmax()]
    return max_accuracy_threshold

# RANDOM FOREST THRESHOLD OPTIMIZATION

In [ ]:
def labels_split(y, X):

    y_pos_1 = []
    y_pos_2 = []
    y_middle_pos = []
    y_pos_Cter = []
    counter = 0
    #print(y)
    for i in range(X.shape[0]):
        length = X[i].shape[0]
        y_pos_1.append(y[counter])
        y_pos_2.append(y[counter + 1])
        y_pos_Cter.append(y[counter + length - 1])
        y_middle_pos.append(y[counter + 2:counter + length - 1].tolist())
        #print(length)
        #print(y[counter:counter + length])
        #print(y[counter])
        #print(y[counter + 1])
        #print(y[counter + length - 1])
        #print(y[(counter + 2):(counter + length - 1)])
        #input()
        counter += length
    y_middle_pos = [x for xs in y_middle_pos for x in xs] # Flatten list
    
    return y_pos_1, y_pos_2, y_middle_pos, y_pos_Cter

def label_reconstruct(y_pos_1, y_pos_2, y_middle_pos, y_pos_Cter, X):
    
    y = []
    counter = 0
    for i in range(X.shape[0]):
        length = X[i].shape[0]
        reconstructed_list = [[y_pos_1[i]], 
                              [y_pos_2[i]], 
                              y_middle_pos[counter:counter + length - 3], 
                              [y_pos_Cter[i]]]
        reconstructed_list = [x for xs in reconstructed_list for x in xs] # Flatten list
        y.append(reconstructed_list)
        counter += length - 3
    
    return [x for xs in y for x in xs]

def whole_rf_predictions(y_test, preds, X):
    counter = 0
    correct = 0
    for i in range(X.shape[0]):
        length = X[i].shape[0]
        correct += np.array_equal(y_test[counter:(counter + length)], preds[counter:(counter + length)])
        #print(y_test[counter:(counter + length)])
        #print(preds[counter:(counter + length)])
        #input()
        counter += length
    return 100*correct/X.shape[0]

In [ ]:
rf_preds = {}
rf_val_preds = {}
rf_thres_preds = {}
rf_correct_predictions = []
rf_jaccard_predictions = []

labels = {}
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for i, (train_index, test_index) in enumerate(tqdm(kfold.split(X))):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    peptides, MHCs = np.array(peptide_list)[test_index], np.array(MHC_list)[test_index]
    
    X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(X_train, y_train, 
                                                                              test_size=0.1, random_state=42)
    
    #print(X_train_train.shape)
    #print(X_train_val.shape)
    #print(y_train_train.shape)
    #print(y_train_val.shape)
    
    # Train Random Forest
    rf = RandomForestClassifier(n_estimators=500, random_state=0, oob_score=True)
    rf.fit(np.vstack(X_train_train), np.hstack(y_train_train))
    
    rf_val_preds[i] = rf.predict_proba(np.vstack(X_train_val))[:, 1]

    y_pos_1, y_pos_2, y_middle_pos, y_pos_Cter = labels_split(np.hstack(y_train_val), X_train_val)
    y_pos_1_pred, y_pos_2_pred, y_middle_pos_pred, y_pos_Cter_pred = labels_split(rf_val_preds[i], X_train_val)
    
    Youden_pos_1 = youden_func(np.array(y_pos_1), np.array(y_pos_1_pred))
    Youden_pos_2 = youden_func(np.array(y_pos_2), np.array(y_pos_2_pred))
    Youden_middle_pos = youden_func(np.array(y_middle_pos), np.array(y_middle_pos_pred))
    Youden_Cter_pos = youden_func(np.array(y_pos_Cter), np.array(y_pos_Cter_pred))
    
    ABS_pos_1 = abs_d_sens_spec_func(np.array(y_pos_1), np.array(y_pos_1_pred))
    ABS_pos_2 = abs_d_sens_spec_func(np.array(y_pos_2), np.array(y_pos_2_pred))
    ABS_middle_pos = abs_d_sens_spec_func(np.array(y_middle_pos), np.array(y_middle_pos_pred))
    ABS_Cter_pos = abs_d_sens_spec_func(np.array(y_pos_Cter), np.array(y_pos_Cter_pred))
    
    ROC01_pos_1 = roc01_func(np.array(y_pos_1), np.array(y_pos_1_pred))
    ROC01_pos_2 = roc01_func(np.array(y_pos_2), np.array(y_pos_2_pred))
    ROC01_middle_pos = roc01_func(np.array(y_middle_pos), np.array(y_middle_pos_pred))
    ROC01_Cter_pos = roc01_func(np.array(y_pos_Cter), np.array(y_pos_Cter_pred))
    
    Acc_pos_1 = accuracy_func(np.array(y_pos_1), np.array(y_pos_1_pred))
    Acc_pos_2 = accuracy_func(np.array(y_pos_2), np.array(y_pos_2_pred))
    Acc_middle_pos = accuracy_func(np.array(y_middle_pos), np.array(y_middle_pos_pred))
    Acc_Cter_pos = accuracy_func(np.array(y_pos_Cter), np.array(y_pos_Cter_pred))
    
    Cohen_pos_1 = cohen_kappa_func(np.array(y_pos_1), np.array(y_pos_1_pred))
    Cohen_pos_2 = cohen_kappa_func(np.array(y_pos_2), np.array(y_pos_2_pred))
    Cohen_middle_pos = cohen_kappa_func(np.array(y_middle_pos), np.array(y_middle_pos_pred))
    Cohen_Cter_pos = cohen_kappa_func(np.array(y_pos_Cter), np.array(y_pos_Cter_pred))

    F1_pos_1 = f1_func(np.array(y_pos_1), np.array(y_pos_1_pred))
    F1_pos_2 = f1_func(np.array(y_pos_2), np.array(y_pos_2_pred))
    F1_middle_pos = f1_func(np.array(y_middle_pos), np.array(y_middle_pos_pred))
    F1_Cter_pos = f1_func(np.array(y_pos_Cter), np.array(y_pos_Cter_pred))

    DOP_pos_1 = DOP_func(np.array(y_pos_1), np.array(y_pos_1_pred))
    DOP_pos_2 = DOP_func(np.array(y_pos_2), np.array(y_pos_2_pred))
    DOP_middle_pos = DOP_func(np.array(y_middle_pos), np.array(y_middle_pos_pred))
    DOP_Cter_pos = DOP_func(np.array(y_pos_Cter), np.array(y_pos_Cter_pred))
    
    rf_preds[i] = rf.predict_proba(np.vstack(X_test))[:, 1]
    y_pos_1, y_pos_2, y_middle_pos, y_pos_Cter = labels_split(np.hstack(y_test), X_test)
    y_pos_1_pred, y_pos_2_pred, y_middle_pos_pred, y_pos_Cter_pred = labels_split(rf_preds[i], X_test)
    
    #print("Round ", i + 1)
    #print("Threshold for position 1: ", optimized_pos_1)
    #print("Accuracy with default threshold: ", accuracy_score(np.array(y_pos_1), np.where(np.array(y_pos_1_pred) >= 0.5, 1, 0)))
    #print("Accuracy with optimized threshold: ", accuracy_score(np.array(y_pos_1), np.where(np.array(y_pos_1_pred) >= optimized_pos_1, 1, 0)))
    #print('-----')
    #print("Threshold for position 2: ", optimized_pos_2)
    #print("Accuracy with default threshold: ", accuracy_score(np.array(y_pos_2), np.where(np.array(y_pos_2_pred) >= 0.5, 1, 0)))
    #print("Accuracy with optimized threshold: ", accuracy_score(np.array(y_pos_2), np.where(np.array(y_pos_2_pred) >= optimized_pos_2, 1, 0)))
    #print('-----')
    #print("Threshold for middle positions 2: ", optimized_middle_pos)
    #print("Accuracy with default threshold: ", accuracy_score(np.array(y_middle_pos), np.where(np.array(y_middle_pos_pred) >= 0.5, 1, 0)))
    #print("Accuracy with optimized threshold: ", accuracy_score(np.array(y_middle_pos), np.where(np.array(y_middle_pos_pred) >= optimized_middle_pos, 1, 0)))
    #print('-----')
    #print("Threshold for C-terminus positions: ", optimized_Cter_pos)
    #print("Accuracy with default threshold: ", accuracy_score(np.array(y_pos_Cter), np.where(np.array(y_pos_Cter_pred) >= 0.5, 1, 0)))
    #print("Accuracy with optimized threshold: ", accuracy_score(np.array(y_pos_Cter), np.where(np.array(y_pos_Cter_pred) >= optimized_Cter_pos, 1, 0)))
    #print('-----')
    
    y_true = label_reconstruct(y_pos_1, y_pos_2, y_middle_pos, y_pos_Cter, X_test)
    y_score = label_reconstruct(np.where(np.array(y_pos_1_pred) >= 0.5, 1, 0),
                                np.where(np.array(y_pos_2_pred) >= 0.5, 1, 0),
                                np.where(np.array(y_middle_pos_pred) >= 0.5, 1, 0), 
                                np.where(np.array(y_pos_Cter_pred) >= 0.5, 1, 0), 
                                X_test)
    y_score_Youden = label_reconstruct(np.where(np.array(y_pos_1_pred) >= Youden_pos_1, 1, 0), 
                                       np.where(np.array(y_pos_2_pred) >= Youden_pos_2, 1, 0), 
                                       np.where(np.array(y_middle_pos_pred) >= Youden_middle_pos, 1, 0), 
                                       np.where(np.array(y_pos_Cter_pred) >= Youden_Cter_pos, 1, 0), 
                                       X_test)
    print("Whole accuracy with default threshold: ", whole_rf_predictions(y_true, y_score, X_test))
    print("Whole accuracy with Youden threshold: ", whole_rf_predictions(y_true, y_score_Youden, X_test))
    
    print('---------------------------')
    input()

In [20]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

svm_preds = {}
svm_correct_predictions = []
svm_jaccard_predictions = []

rf_preds = {}
rf_train_preds = {}
rf_train_proba_preds = {}
rf_correct_predictions = []
rf_jaccard_predictions = []

rf_models = {}
data_points = {}

crf_preds = {}
crf_correct_predictions = []
crf_jaccard_predictions = []

#sper_preds = {}
#sper_correct_predictions = []
#sper_jaccard_predictions = []

NetMHCpan_correct_predictions = []
NetMHCpan_jaccard_predictions = []

labels = {}

for i, (train_index, test_index) in enumerate(tqdm(kfold.split(X))):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    peptides, MHCs = np.array(peptide_list)[test_index], np.array(MHC_list)[test_index]
    
    # Train linear SVM
    svm = LinearSVC(dual=False, C=.1)
    svm.fit(np.vstack(X_train), np.hstack(y_train))
    svm_preds[i] = svm.predict(np.vstack(X_test))
    svm_correct_predictions.append(correct_predictions(y_test, svm_preds[i]))
    svm_jaccard_predictions.append(jaccard_predictions(y_test, svm_preds[i]))
    
    # Train Random Forest
    rf = RandomForestClassifier(n_estimators=500, random_state=0, oob_score=True)
    rf.fit(np.vstack(X[train_index]), np.hstack(y[train_index]))
    
    print(np.hstack(y[train_index]))
    print(np.vstack(X[train_index]))
    print(np.hstack(y[train_index]).shape)
    print(np.vstack(X[train_index]).shape)
    rf_preds[i] = rf.predict(np.vstack(X_test))
    rf_train_preds[i] = rf.predict(np.vstack(X[train_index]))
    rf_train_proba_preds[i] = rf.predict_proba(np.vstack(X[train_index]))
    rf_correct_predictions.append(correct_predictions(y_test, rf_preds[i]))
    correct_rf_predictions(y[train_index], rf_train_preds[i], rf_train_proba_preds[i], 
                                                         np.array(peptide_list)[train_index],
                                                         np.array(MHC_list)[train_index],
                                                         X[train_index])
    rf_jaccard_predictions.append(jaccard_predictions(y_test, rf_preds[i]))
    
    rf_models[i] = rf
    data_points[i] = np.apply_along_axis(join_txt, 0, [np.array(MHC_list)[train_index], 
                                                       np.array(peptide_list)[train_index]])
    
    # Train linear chain CRF
    model = ChainCRF()
    crf = FrankWolfeSSVM(model=model, C=.1, max_iter=20)
    crf.fit(X_train, y_train)
    crf_preds[i] = crf.predict(X_test)
    crf_correct_predictions.append(correct_predictions(y_test, np.hstack(crf_preds[i])))
    crf_jaccard_predictions.append(jaccard_predictions(y_test, np.hstack(crf_preds[i])))
    
    # Train Structured perceptron
    #sper = StructuredPerceptron(model=model, max_iter=500)
    #sper.fit(X_train, y_train)
    #sper_preds[i] = sper.predict(X_test)
    #sper_correct_predictions.append(correct_predictions(y_test, np.hstack(sper_preds[i])))
    #sper_jaccard_predictions.append(jaccard_predictions(y_test, np.hstack(sper_preds[i])))
    
    y_test_NetMHcpan41 = y_NetMHCpan[test_index]
    NetMHCpan_correct_predictions.append(correct_predictions(y_test, np.hstack(y_test_NetMHcpan41)))
    NetMHCpan_jaccard_predictions.append(jaccard_predictions(y_test, np.hstack(y_test_NetMHcpan41)))
    
    labels[i] =  np.hstack(y_test)

[0 1 0 ... 0 0 1]
[[0.    0.002 0.   ]
 [0.026 0.009 0.045]
 [0.282 0.091 0.079]
 ...
 [0.004 0.001 0.   ]
 [0.035 0.017 0.   ]
 [0.02  0.196 0.   ]]
(4508,)
(4508, 3)
DMANVSTGR BoLA-2*012:01
[0 1 0 0 0 0 0 0 1]
[1 1 0 0 0 0 0 0 1]
[[0.32790317 0.67209683]
 [0.109      0.891     ]
 [0.98733333 0.01266667]
 [0.998      0.002     ]
 [1.         0.        ]
 [0.998      0.002     ]
 [0.99466667 0.00533333]
 [1.         0.        ]
 [0.126      0.874     ]]
[[0.    0.002 0.   ]
 [0.026 0.009 0.045]
 [0.282 0.091 0.079]
 [0.052 0.041 0.04 ]
 [0.019 0.083 0.097]
 [0.066 0.07  0.08 ]
 [0.097 0.089 0.101]
 [0.036 0.031 0.   ]
 [0.038 0.092 0.   ]]
RMANVSTGR BoLA-2*012:01
[0 1 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 1]
[[0.78786667 0.21213333]
 [0.109      0.891     ]
 [0.98733333 0.01266667]
 [0.998      0.002     ]
 [1.         0.        ]
 [0.998      0.002     ]
 [0.99466667 0.00533333]
 [1.         0.        ]
 [0.126      0.874     ]]
[[0.    0.124 0.   ]
 [0.026 0.009 0.045]
 [0.282 0.091 0.079]

 [0.055 0.41  0.   ]]
LYKKLKREITF HLA-A*24:02
[0 1 0 0 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 0 0 1]
[[0.971      0.029     ]
 [0.0044023  0.9955977 ]
 [0.974      0.026     ]
 [1.         0.        ]
 [0.99       0.01      ]
 [0.982      0.018     ]
 [1.         0.        ]
 [1.         0.        ]
 [0.99733333 0.00266667]
 [0.94       0.06      ]
 [0.         1.        ]]
[[0.    0.101 0.012]
 [0.045 0.649 0.037]
 [0.    0.035 0.038]
 [0.035 0.038 0.049]
 [0.064 0.097 0.084]
 [0.049 0.065 0.057]
 [0.049 0.043 0.032]
 [0.074 0.06  0.064]
 [0.06  0.062 0.053]
 [0.049 0.095 0.   ]
 [0.037 0.395 0.   ]]
LYKKLKREMTF HLA-A*24:02
[0 1 0 0 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 0 0 1]
[[0.971     0.029    ]
 [0.0044023 0.9955977]
 [0.974     0.026    ]
 [1.        0.       ]
 [0.99      0.01     ]
 [0.982     0.018    ]
 [1.        0.       ]
 [1.        0.       ]
 [0.986     0.014    ]
 [0.94      0.06     ]
 [0.        1.       ]]
[[0.    0.101 0.012]
 [0.045 0.649 0.037]
 [0.    0.035 0.038]
 [0.035 0

[0 1 0 ... 0 0 1]
[[0.    0.124 0.   ]
 [0.026 0.009 0.045]
 [0.282 0.091 0.079]
 ...
 [0.08  0.072 0.058]
 [0.044 0.014 0.   ]
 [0.044 0.148 0.   ]]
(4512,)
(4512, 3)
RMANVSTGR BoLA-2*012:01
[0 1 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 1]
[[0.728      0.272     ]
 [0.34133333 0.65866667]
 [0.95415238 0.04584762]
 [1.         0.        ]
 [0.97353333 0.02646667]
 [1.         0.        ]
 [0.99       0.01      ]
 [0.996      0.004     ]
 [0.234      0.766     ]]
[[0.    0.124 0.   ]
 [0.026 0.009 0.045]
 [0.282 0.091 0.079]
 [0.052 0.041 0.04 ]
 [0.019 0.083 0.097]
 [0.066 0.07  0.08 ]
 [0.097 0.089 0.101]
 [0.036 0.031 0.   ]
 [0.038 0.092 0.   ]]
IPAYGVLTI BoLA-2*018:01
[0 1 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 1]
[[0.97581587 0.02418413]
 [0.02942576 0.97057424]
 [0.962      0.038     ]
 [0.96       0.04      ]
 [0.998      0.002     ]
 [1.         0.        ]
 [0.86530556 0.13469444]
 [0.662      0.338     ]
 [0.002      0.998     ]]
[[0.    0.07  0.001]
 [0.    0.729 0.079]
 [0.218 0.094 0.076]

[1 1 0 0 0 0 0 0 1]
[1 1 0 0 0 0 0 0 1]
[[0.092      0.908     ]
 [0.35200303 0.64799697]
 [0.968      0.032     ]
 [0.9535     0.0465    ]
 [0.994      0.006     ]
 [0.996      0.004     ]
 [0.991      0.009     ]
 [0.998      0.002     ]
 [0.         1.        ]]
[[0.    0.121 0.098]
 [0.057 0.109 0.069]
 [0.    0.123 0.056]
 [0.033 0.031 0.039]
 [0.105 0.064 0.084]
 [0.064 0.084 0.081]
 [0.027 0.027 0.022]
 [0.059 0.072 0.   ]
 [0.054 0.892 0.   ]]
ATIGTAMYK HLA-A*11:01
[1 1 0 0 0 0 0 0 1]
[1 1 0 0 0 0 0 0 1]
[[0.092 0.908]
 [0.018 0.982]
 [0.994 0.006]
 [0.996 0.004]
 [1.    0.   ]
 [1.    0.   ]
 [0.991 0.009]
 [0.954 0.046]
 [0.    1.   ]]
[[0.    0.121 0.098]
 [0.07  0.214 0.031]
 [0.109 0.069 0.033]
 [0.03  0.064 0.054]
 [0.063 0.072 0.074]
 [0.076 0.084 0.081]
 [0.027 0.027 0.022]
 [0.053 0.046 0.008]
 [0.054 0.892 0.   ]]
GTSGSPIINR HLA-A*11:01
[1 1 0 0 0 0 0 0 0 1]
[1 1 0 0 0 0 0 0 0 1]
[[0.02139524 0.97860476]
 [0.01       0.99      ]
 [0.988      0.012     ]
 [0.99164444 0

[0 1 0 ... 0 0 1]
[[0.    0.002 0.   ]
 [0.026 0.009 0.045]
 [0.282 0.091 0.079]
 ...
 [0.004 0.001 0.   ]
 [0.035 0.017 0.   ]
 [0.02  0.196 0.   ]]
(4509,)
(4509, 3)
DMANVSTGR BoLA-2*012:01
[0 1 0 0 0 0 0 0 1]
[1 1 0 0 0 0 0 0 1]
[[0.33250952 0.66749048]
 [0.1        0.9       ]
 [0.982      0.018     ]
 [0.992      0.008     ]
 [0.994      0.006     ]
 [1.         0.        ]
 [0.99693333 0.00306667]
 [1.         0.        ]
 [0.1036     0.8964    ]]
[[0.    0.002 0.   ]
 [0.026 0.009 0.045]
 [0.282 0.091 0.079]
 [0.052 0.041 0.04 ]
 [0.019 0.083 0.097]
 [0.066 0.07  0.08 ]
 [0.097 0.089 0.101]
 [0.036 0.031 0.   ]
 [0.038 0.092 0.   ]]
RMANVSTGR BoLA-2*012:01
[0 1 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 1]
[[0.782      0.218     ]
 [0.1        0.9       ]
 [0.982      0.018     ]
 [0.992      0.008     ]
 [0.994      0.006     ]
 [1.         0.        ]
 [0.99693333 0.00306667]
 [1.         0.        ]
 [0.1036     0.8964    ]]
[[0.    0.124 0.   ]
 [0.026 0.009 0.045]
 [0.282 0.091 0.079]

...

 [0.055 0.41  0.   ]]
RYGFVANF HLA-A*24:02
[0 1 0 0 1 0 0 1]
[0 1 0 0 1 0 0 1]
[[0.99795982 0.00204018]
 [0.006      0.994     ]
 [0.88       0.12      ]
 [0.979      0.021     ]
 [0.354      0.646     ]
 [0.998      0.002     ]
 [0.99       0.01      ]
 [0.012      0.988     ]]
[[0.         0.04604605 0.        ]
 [0.0960961  0.65965966 0.06006006]
 [0.         0.06806807 0.03703704]
 [0.06306306 0.03303303 0.06706707]
 [0.08008008 0.07307307 0.05305305]
 [0.08108108 0.06206206 0.06306306]
 [0.02702703 0.03603604 0.        ]
 [0.05705706 0.41141141 0.        ]]
RYPLTFGW HLA-A*24:02
[0 1 0 0 0 0 0 1]
[0 1 0 0 0 0 0 1]
[[0.99795982 0.00204018]
 [0.006      0.994     ]
 [0.972      0.028     ]
 [0.996      0.004     ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [0.074      0.926     ]]
[[0.         0.04604605 0.        ]
 [0.0960961  0.65965966 0.06006006]
 [0.         0.13313313 0.09009009]
 [0.12312312 0.10510511 0.10910911]
 [0.05605606 0.05005005 0.063

[0 1 0 ... 0 0 1]
[[0.    0.002 0.   ]
 [0.026 0.009 0.045]
 [0.282 0.091 0.079]
 ...
 [0.004 0.001 0.   ]
 [0.035 0.017 0.   ]
 [0.02  0.196 0.   ]]
(4511,)
(4511, 3)
DMANVSTGR BoLA-2*012:01
[0 1 0 0 0 0 0 0 1]
[1 1 0 0 0 0 0 0 1]
[[0.33855317 0.66144683]
 [0.317      0.683     ]
 [0.93983333 0.06016667]
 [0.994      0.006     ]
 [0.9735     0.0265    ]
 [1.         0.        ]
 [0.994      0.006     ]
 [0.998      0.002     ]
 [0.24230952 0.75769048]]
[[0.    0.002 0.   ]
 [0.026 0.009 0.045]
 [0.282 0.091 0.079]
 [0.052 0.041 0.04 ]
 [0.019 0.083 0.097]
 [0.066 0.07  0.08 ]
 [0.097 0.089 0.101]
 [0.036 0.031 0.   ]
 [0.038 0.092 0.   ]]
IPAYGVLTI BoLA-2*018:01
[0 1 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 1]
[[0.99436571 0.00563429]
 [0.06293333 0.93706667]
 [0.906      0.094     ]
 [0.972      0.028     ]
 [1.         0.        ]
 [0.996      0.004     ]
 [0.80703571 0.19296429]
 [0.67       0.33      ]
 [0.004      0.996     ]]
[[0.    0.07  0.001]
 [0.    0.729 0.079]
 [0.218 0.094 0.076]

 [0.         1.        ]]
[[0.    0.077 0.   ]
 [0.046 0.195 0.065]
 [0.    0.095 0.114]
 [0.136 0.066 0.103]
 [0.06  0.06  0.075]
 [0.057 0.055 0.073]
 [0.07  0.078 0.045]
 [0.024 0.013 0.01 ]
 [0.008 0.004 0.   ]
 [0.055 0.41  0.   ]]
RYGFVANF HLA-A*24:02
[0 1 0 0 1 0 0 1]
[0 1 0 0 1 0 0 1]
[[0.99064567 0.00935433]
 [0.014      0.986     ]
 [0.89       0.11      ]
 [0.99       0.01      ]
 [0.346      0.654     ]
 [0.994      0.006     ]
 [0.982      0.018     ]
 [0.006      0.994     ]]
[[0.         0.04604605 0.        ]
 [0.0960961  0.65965966 0.06006006]
 [0.         0.06806807 0.03703704]
 [0.06306306 0.03303303 0.06706707]
 [0.08008008 0.07307307 0.05305305]
 [0.08108108 0.06206206 0.06306306]
 [0.02702703 0.03603604 0.        ]
 [0.05705706 0.41141141 0.        ]]
RYPLTFGW HLA-A*24:02
[0 1 0 0 0 0 0 1]
[0 1 0 0 0 0 0 1]
[[0.99064567 0.00935433]
 [0.014      0.986     ]
 [0.956      0.044     ]
 [0.996      0.004     ]
 [0.998      0.002     ]
 [0.996      0.004     ]
 [1.     

[0 1 0 ... 0 0 1]
[[0.    0.002 0.   ]
 [0.026 0.009 0.045]
 [0.282 0.091 0.079]
 ...
 [0.004 0.001 0.   ]
 [0.035 0.017 0.   ]
 [0.02  0.196 0.   ]]
(4517,)
(4517, 3)
DMANVSTGR BoLA-2*012:01
[0 1 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 1]
[[0.5589381 0.4410619]
 [0.1055    0.8945   ]
 [0.984     0.016    ]
 [0.996     0.004    ]
 [0.986     0.014    ]
 [0.998     0.002    ]
 [0.998     0.002    ]
 [1.        0.       ]
 [0.086     0.914    ]]
[[0.    0.002 0.   ]
 [0.026 0.009 0.045]
 [0.282 0.091 0.079]
 [0.052 0.041 0.04 ]
 [0.019 0.083 0.097]
 [0.066 0.07  0.08 ]
 [0.097 0.089 0.101]
 [0.036 0.031 0.   ]
 [0.038 0.092 0.   ]]
RMANVSTGR BoLA-2*012:01
[0 1 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 1]
[[0.76733333 0.23266667]
 [0.1055     0.8945    ]
 [0.984      0.016     ]
 [0.996      0.004     ]
 [0.986      0.014     ]
 [0.998      0.002     ]
 [0.998      0.002     ]
 [1.         0.        ]
 [0.086      0.914     ]]
[[0.    0.124 0.   ]
 [0.026 0.009 0.045]
 [0.282 0.091 0.079]
 [0.052 0.041 0.0

 [0.         1.        ]]
[[0.    0.145 0.131]
 [0.145 0.131 0.095]
 [0.    0.009 0.002]
 [0.095 0.103 0.079]
 [0.103 0.079 0.09 ]
 [0.008 0.009 0.005]
 [0.075 0.065 0.058]
 [0.174 0.145 0.139]
 [0.077 0.07  0.   ]
 [0.057 0.913 0.   ]]
TIAMELIRMIK HLA-A*11:01
[1 1 0 0 0 0 0 0 0 0 1]
[1 1 0 0 0 0 0 0 0 0 1]
[[0.012   0.988  ]
 [0.2215  0.7785 ]
 [0.998   0.002  ]
 [0.93    0.07   ]
 [0.996   0.004  ]
 [0.99825 0.00175]
 [0.998   0.002  ]
 [0.984   0.016  ]
 [0.992   0.008  ]
 [0.992   0.008  ]
 [0.      1.     ]]
[[0.    0.062 0.25 ]
 [0.047 0.108 0.09 ]
 [0.082 0.086 0.08 ]
 [0.029 0.004 0.018]
 [0.111 0.062 0.073]
 [0.068 0.067 0.105]
 [0.036 0.054 0.063]
 [0.03  0.035 0.031]
 [0.018 0.034 0.018]
 [0.079 0.038 0.   ]
 [0.069 0.852 0.   ]]
TMVMELIRMIK HLA-A*11:01
[1 0 0 0 0 0 0 0 0 0 1]
[1 0 0 0 0 0 0 0 0 0 1]
[[0.012      0.988     ]
 [0.97       0.03      ]
 [0.97369255 0.02630745]
 [0.93       0.07      ]
 [0.996      0.004     ]
 [0.99825    0.00175   ]
 [0.998      0.002     ]
 [

In [21]:
print('Exact Accuracies: ')
print('Linear SVM: ', str(sum(svm_correct_predictions)/dataset.shape[0]))
print('Random Forest: ', str(sum(rf_correct_predictions)/dataset.shape[0]))
print('Linear CRF: ',  str(sum(crf_correct_predictions)/dataset.shape[0]))
#print('Structured Perceptron: ',  str(sum(sper_correct_predictions)/dataset.shape[0]))
print('NetMHCpan4.1: ',  str(sum(NetMHCpan_correct_predictions)/dataset.shape[0]))

#print('\nJaccard Accuracies: ')
#print('Linear SVM: ', str(sum(svm_jaccard_predictions)/dataset.shape[0]))
#print('Random Forest: ', str(sum(rf_jaccard_predictions)/dataset.shape[0]))
#print('Linear CRF: ',  str(sum(crf_jaccard_predictions)/dataset.shape[0]))
#print('Structured Perceptron: ',  str(sum(sper_jaccard_predictions)/dataset.shape[0]))
#print('NetMHCpan4.1: ',  str(sum(NetMHCpan_jaccard_predictions)/dataset.shape[0]))

Exact Accuracies: 
Linear SVM:  0.21042830540037244
Random Forest:  0.515828677839851
Linear CRF:  0.25884543761638734
NetMHCpan4.1:  0.41899441340782123


In [22]:
non_canonical_list = ['5F7D.pdb', '3GIV.pdb', '5DDH.pdb', '2CLR.pdb', '5EOT.pdb', '5FA4.pdb', 
                      '3RWJ.pdb', '4O2F.pdb', '2GTW.pdb', '2GTW.pdb', '1DUY.pdb', '1S7R.pdb',
                      '2BVQ.pdb', '3RWI.pdb', '4MJ6.pdb']
dataset_val = pd.read_csv("./Anchor_pos_data/Template_information.csv")
dataset_val = dataset_val[(dataset_val['peptide_length'] > 7) & (dataset_val['MHC']).isin(frequencies_alleles)]
dataset_val = dataset_val[~dataset_val['pdb_code'].str.contains("_B.pdb")]
dataset_val["pdb_code"] = dataset_val["pdb_code"].str.replace("_A","")
dataset_val = dataset_val[['pdb_code', 'MHC', 'peptide', 'peptide_length', 'anchors', 'NetMHCpan4.1_anchors']].drop_duplicates()
dataset_val = dataset_val[dataset_val['pdb_code'].isin(non_canonical_list)]
val_MHC_list = dataset_val['MHC'].tolist()
val_peptide_list = dataset_val['peptide'].tolist()
val_labels_list = dataset_val['anchors'].tolist()
val_NetMHCpan_list = dataset_val['NetMHCpan4.1_anchors'].tolist()
val_lengths_list = dataset_val['peptide_length'].tolist()

In [23]:
peptide_MHCs = list(np.apply_along_axis(join_txt, 0, [np.array(dataset_val['MHC']), np.array(dataset_val['peptide'])]))

In [24]:
def extract_positions(position, peptide, feature_size):
    scale = range(-int(np.floor(feature_size/2)), int(np.floor(feature_size/2)) + 1)
    position_list = []
    append_left = 0
    append_right = 0
    for x in scale:
        if (position + x < 1):
            position_list.append(0)
            append_left += 1
        elif (position + x > len(peptide)):
            position_list.append(0)
            append_right += 1
        else:
            position_list.append(position + x)
    return position_list, append_left, append_right

In [25]:
wind = int(np.floor(windows_size/2))
for i, peptide_MHC in enumerate(peptide_MHCs):
    all_features_list = []
    all_labels_list = []
    all_labels_NetMHCpan_list = []
    ranges = list(range(10))
    for j in range(len(data_points)):
        if peptide_MHC in list(data_points[j]):
            ranges.remove(j)
    rf_to_predict = ranges[0]
    
    aa_neighbors = [''.join(w) for w in list(ngrams(val_peptide_list[i], windows_size, pad_left=True, pad_right=True, 
                                                    left_pad_symbol='X', 
                                                    right_pad_symbol='X'))][wind:]
    aa_neighbors = aa_neighbors[:len(aa_neighbors) - wind]
    
    sub_seq_feature_list = []
    for k, sub_seq in enumerate(aa_neighbors):
    
        # Break subsequence down to amino_acids
        pep_sequence = list(sub_seq)
        
        # For each amino-acid, extract frequencies and important positions
        feature_list = []
        for j, amino_acid in enumerate(pep_sequence):
        
            freq_features = frequencies[(frequencies['allele'] == val_MHC_list[i]) & (frequencies['length'] == val_lengths_list[i])]
            position_list, append_left, append_right = extract_positions(j + k + 1 - wind, val_peptide_list[i], feature_size)
            freq_features = freq_features[freq_features['position'].isin(position_list)][amino_acid].values
            freq_features = np.hstack([np.zeros(append_left), freq_features, np.zeros(append_right)])
            feature_list.append(freq_features) 
        
        features = np.array(np.hstack(feature_list))
        #posn_feature = (k == 0)
        #posnplus1_feature = (k == 1)
        #posc_feature = (k == len(peptide) - 1) # Get those out!
        #features = np.hstack([features, [posn_feature], [posnplus1_feature], [posc_feature]])
        sub_seq_feature_list.append(features)
    all_features_list.append(np.array(sub_seq_feature_list, dtype=np.float))
    
    labels_int_list = [int(x) - 1 for x in val_labels_list[i].split(',')]
    labels = np.zeros(len(val_peptide_list[i]), dtype=int)
    labels[labels_int_list] = 1
    all_labels_list.append(labels)
    
    labels_NetMHCpan_int_list = [int(x) - 1 for x in val_NetMHCpan_list[i].split(',')]
    labels_NetMHCpan = np.zeros(len(val_peptide_list[i]), dtype=int)
    labels_NetMHCpan[labels_NetMHCpan_int_list] = 1
    all_labels_NetMHCpan_list.append(labels_NetMHCpan)
    
    X = np.array(all_features_list, dtype=object)
    y = np.array(all_labels_list, dtype=object)
    
    print(peptide_MHC)
    print(X)
    print(y)
    y_test = rf_models[rf_to_predict].predict(np.vstack(X))
    print(y_test)
    y_proba = rf_models[rf_to_predict].predict_proba(np.vstack(X))
    print(y_proba)

H2-Kb-KAVYNLATM
[[[0.0 0.07200000000000001 0.0]
  [0.125 0.179 0.027999999999999997]
  [0.133 0.1 0.065]
  [0.23800000000000002 0.026000000000000002 0.14400000000000002]
  [0.025 0.055999999999999994 0.02]
  [0.085 0.11900000000000001 0.105]
  [0.069 0.09699999999999999 0.08]
  [0.099 0.08900000000000001 0.0]
  [0.013000000000000001 0.08 0.0]]]
[[0 1 1 0 0 0 0 0 1]]
[0 1 0 0 0 0 0 0 1]
[[0.998      0.002     ]
 [0.25514149 0.74485851]
 [0.8754     0.1246    ]
 [0.896      0.104     ]
 [1.         0.        ]
 [0.9405     0.0595    ]
 [0.94021695 0.05978305]
 [0.97933333 0.02066667]
 [0.038      0.962     ]]
HLA-A*02:01-FVLELEPEWTVK
[[[0.0 0.09 0.003]
  [0.055999999999999994 0.094 0.08199999999999999]
  [0.623 0.134 0.016]
  [0.022000000000000002 0.185 0.102]
  [0.016 0.085 0.062]
  [0.102 0.105 0.1]
  [0.076 0.10400000000000001 0.062]
  [0.1 0.09699999999999999 0.057999999999999996]
  [0.01 0.006 0.01]
  [0.045 0.063 0.08]
  [0.095 0.061 0.243]
  [0.055 0.02 0.0]]]
[[0 1 0 0 0 0 0 0 0 

Mamu-B*017:01:01:01-HLEVQGYW
[[[0.0 0.055055055055055056 0.25925925925925924]
  [0.12912912912912913 0.04104104104104104 0.11311311311311313]
  [0.031031031031031032 0.015015015015015015 0.08208208208208208]
  [0.07207207207207207 0.07807807807807808 0.08008008008008008]
  [0.03303303303303303 0.027027027027027032 0.04704704704704705]
  [0.06306306306306306 0.025025025025025027 0.02002002002002002]
  [0.07807807807807808 0.05105105105105105 0.037037037037037035]
  [0.004004004004004004 0.6076076076076076 0.0]]]
[[1 0 0 0 0 0 0 1]]
[1 0 0 0 0 0 0 1]
[[0.108      0.892     ]
 [0.734      0.266     ]
 [0.926      0.074     ]
 [0.982      0.018     ]
 [0.73842857 0.26157143]
 [0.934      0.066     ]
 [0.9814     0.0186    ]
 [0.         1.        ]]


# (windows_size, feature_size) combinations:

(1,1):
- Linear SVM:  0.3817504655493482
- Random Forest:  0.5586592178770949
- Linear CRF:  0.409683426443203
- Structured Perceptron:  0.0633147113594041
- NetMHCpan4.1:  0.41899441340782123

(1,3):
- Linear SVM:  0.35567970204841715
- Random Forest:  0.5996275605214153
- Linear CRF:  0.409683426443203
- Structured Perceptron:  0.13594040968342644
- NetMHCpan4.1:  0.41899441340782123

(1,5):
- Linear SVM:  0.3575418994413408
- Random Forest:  0.6312849162011173
- Linear CRF:  0.409683426443203
- Structured Perceptron:  0.1042830540037244
- NetMHCpan4.1:  0.41899441340782123

(1, 7):
- Linear SVM:  0.35567970204841715
- Random Forest:  0.6443202979515829
- Linear CRF:  0.409683426443203
- Structured Perceptron:  0.0893854748603352
- NetMHCpan4.1:  0.41899441340782123

(1, 9):
- Linear SVM:  0.35940409683426444
- Random Forest:  0.6350093109869647
- Linear CRF:  0.409683426443203
- Structured Perceptron:  0.22532588454376165
- NetMHCpan4.1:  0.41899441340782123

--------------------------
(3, 1):
- Linear SVM:  0.3929236499068901
- Random Forest:  0.5865921787709497
- Linear CRF:  0.409683426443203
- Structured Perceptron:  0.05772811918063315
- NetMHCpan4.1:  0.41899441340782123

(3, 3):
- Linear SVM:  0.3929236499068901
- Random Forest:  0.6182495344506518
- Linear CRF:  0.409683426443203
- Structured Perceptron:  0.2681564245810056
- NetMHCpan4.1:  0.41899441340782123

(3, 5):
- Linear SVM:  0.3854748603351955
- Random Forest:  0.6443202979515829
- Linear CRF:  0.409683426443203
- Structured Perceptron:  0.09869646182495345
- NetMHCpan4.1:  0.41899441340782123

(3, 7):
- Linear SVM:  0.3817504655493482
- Random Forest:  0.6368715083798883
- Linear CRF:  0.409683426443203
- Structured Perceptron:  0.12476722532588454
- NetMHCpan4.1:  0.41899441340782123

(3, 9):
- Linear SVM:  0.3817504655493482
- Random Forest:  0.62756052141527
- Linear CRF:  0.409683426443203
- Structured Perceptron:  0.21042830540037244
- NetMHCpan4.1:  0.41899441340782123

-------------------------
(5, 1):
- Linear SVM:  0.3947858472998138
- Random Forest:  0.5623836126629422
- Linear CRF:  0.409683426443203
- Structured Perceptron:  0.21042830540037244
- NetMHCpan4.1:  0.41899441340782123

(5, 3):
- Linear SVM:  0.38733705772811916
- Random Forest:  0.6163873370577281
- Linear CRF:  0.409683426443203
- Structured Perceptron:  0.16387337057728119
- NetMHCpan4.1:  0.41899441340782123

(5, 5):
- Linear SVM:  0.3817504655493482
- Random Forest:  0.6201117318435754
- Linear CRF:  0.409683426443203
- Structured Perceptron:  0.1973929236499069
- NetMHCpan4.1:  0.41899441340782123

(5, 7):
- Linear SVM:  0.3854748603351955
- Random Forest:  0.6182495344506518
- Linear CRF:  0.409683426443203
- Structured Perceptron:  0.148975791433892
- NetMHCpan4.1:  0.41899441340782123

(5, 9):
- Linear SVM:  0.37988826815642457
- Random Forest:  0.6238361266294227
- Linear CRF:  0.409683426443203
- Structured Perceptron:  0.14152700186219738
- NetMHCpan4.1:  0.41899441340782123

--------------------------

# SINGLE CELL EXECUTION

In [17]:
for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

ValueError: Cannot have number of splits n_splits=10 greater than the number of samples: n_samples=1.

In [ ]:
def correct_predictions(y_test, preds):
    counter = 0
    correct = 0
    for i in range(y_test.shape[0]):
        correct += np.array_equal(y_test[i], preds[counter:(counter + y_test[i].shape[0])])
        counter += y_test[i].shape[0]

    print(str(correct) + "/" + str(y_test.shape[0]) + " correct predictions!")

In [ ]:
# Train linear SVM
svm = LinearSVC(dual=False, C=.1)

# flatten input
svm.fit(np.vstack(X_train), np.hstack(y_train))

print("Test score with linear SVM: %f" % svm.score(np.vstack(X_test), np.hstack(y_test)))

preds = svm.predict(np.vstack(X_test))
correct_predictions(y_test, preds)

In [ ]:
clf = RandomForestClassifier(n_estimators=1000, random_state=0)

clf.fit(np.vstack(X_train), np.hstack(y_train))

print("Test score with Random Forest: %f" % clf.score(np.vstack(X_test), np.hstack(y_test)))

preds = clf.predict(np.vstack(X_test))
correct_predictions(y_test, preds)

In [ ]:
test2 = list(zip(np.array(peptide_list)[list(test_index)].tolist(),
                 np.array(MHC_list)[list(test_index)].tolist()))
zipped_list = test2[:]
zipped_list_2 = np.array(test2)

counter = 0
for i in range(y_test.shape[0]):
    print(zipped_list_2[i])
    print(y_test[i])
    print(preds[counter:(counter + y_test[i].shape[0])])
    #input()
    counter += y_test[i].shape[0]

In [ ]:
feature_names = [f"feature {i}" for i in range(X[0].shape[1])]
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_], axis=0)
forest_importances = pd.Series(importances, index=feature_names)
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
result = permutation_importance(
    clf, np.vstack(X_test), np.hstack(y_test), n_repeats=10, random_state=42, n_jobs=2
)
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.tight_layout()
plt.show()

In [ ]:
# Train linear chain CRF
model = ChainCRF()
ssvm = FrankWolfeSSVM(model=model, C=.1, max_iter=20)
ssvm.fit(X_train, y_train)

print("Test score with chain CRF: %f" % ssvm.score(X_test, y_test))

preds = ssvm.predict(X_test)
correct_predictions(y_test, np.hstack(preds))

In [ ]:
ssvm = OneSlackSSVM(model=model, C=.1)
ssvm.fit(X_train, y_train)

print("Test score with chain CRF: %f" % ssvm.score(X_test, y_test))

preds = ssvm.predict(X_test)
correct_predictions(y_test, np.hstack(preds))

In [ ]:
ssvm = NSlackSSVM(model=model, C=.1)
ssvm.fit(X_train, y_train)

print("Test score with chain CRF: %f" % ssvm.score(X_test, y_test))

preds = ssvm.predict(X_test)
correct_predictions(y_test, np.hstack(preds))